<a href="https://colab.research.google.com/github/dinhan2411/HDH-Ass2/blob/master/Visualization/visualKG_with_neo4j.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dùng neo4j
- tạo database neo4j
- kết nối với neo4j sandbox bằng connection
- tạo các entity + relation bằng các command trong notebook
- vào neo4j browser (desktop) để xem graph

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd "/content/gdrive/MyDrive/Colab/Neural_LP"
!git clone https://github.com/fanyangxyz/Neural-LP.git

In [5]:
!pip install neo4j
from neo4j import GraphDatabase

     |████████████████████████████████| 81kB 3.8MB/s 
  Created wheel for neo4j: filename=neo4j-4.3.1-cp37-none-any.whl size=99332 sha256=e7fc20c188291b49a874c543be6fd35038ae9d1da8400afa706b4303930e0f08
  Stored in directory: /root/.cache/pip/wheels/23/13/72/0cc2405898bd9a7baef6512df3abf83873da9ba48c04acc818
Successfully built neo4j


Class Connection

In [6]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

Tạo connection cố định

In [7]:
conn = Neo4jConnection(uri="bolt://3.83.36.217:7687", user="neo4j", pwd="rides-regulations-anthem")

Chọn toàn bộ Entity

In [ ]:
conn.query("match(n) return(n)")

Xóa toàn bộ Entity + Relation

In [ ]:
conn.query("MATCH (n) DETACH DELETE n")

Tạo graph từ file facts.txt

In [ ]:
%cd "/content/gdrive/MyDrive/Colab/Neural_LP/Neural-LP/datasets/family"

entities = []
f = open("facts.txt", "r")
for x in range(3000):
  line = f.readline()
  spl = line.split()
  if spl[0] not in entities:
    entities.append(spl[0])
    queryy = "CREATE ( e{} :Entity {{ name:'{}' }})".format(str(spl[0].strip()), str(spl[0].strip()))
    respond = conn.query(queryy)
    print(respond)
  if spl[2] not in entities:
    entities.append(spl[2])
    queryy = "CREATE ( e{} :Entity {{ name:'{}' }})".format(str(spl[2].strip()), str(spl[2].strip()))
    respond = conn.query(queryy)
    print(respond)
  
  queryy = "MATCH (e{a}:Entity), (e{b}:Entity) WHERE e{a}.name = '{a}' AND e{b}.name = '{b}' CREATE (e{a})-[{rel}:{rel}]->(e{b})".format(
      a = str(spl[0].strip()), b = str(spl[2].strip()), rel = str(spl[1].strip()))
  respond = conn.query(queryy)
  print(respond)
  print(x)
f.close()

Tạo graph từ file entities.txt và facts.txt

In [ ]:
%cd "/content/gdrive/MyDrive/Colab/Neural_LP/Neural-LP/datasets/family"
# count = 0
# print("Using readlines()")

conn.query("MATCH (n) DETACH DELETE n")

f = open("entities.txt", "r")
for line in f:
  line = line.strip()
  queryy = "CREATE ( e{} :Entity {{ name:'{}' }})".format(str(line), str(line))
  respond = conn.query(queryy)
  print(respond)
f.close()

f = open("facts.txt", "r")
for line in f:
  print(line)
  spl = line.split()

  queryy = "MATCH (e{a}:Entity), (e{b}:Entity) WHERE e{a}.name = '{a}' AND e{b}.name = '{b}' CREATE (e{a})-[{rel}:{rel}]->(e{b})".format(
      a = str(spl[0].strip()), b = str(spl[2].strip()), rel = str(spl[1].strip()))
  respond = conn.query(queryy)
  print(respond)
  #print(x)
f.close()